In [ ]:
#start date: 24/01/2024
#project_title: An End-to-End Project on Time Series Analysis and Forecasting with Python
#project_link: 

import warnings
import itertools
import numpy as np
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')
import pandas as pd
import statsmodels.api as sm

#reading and viewing dataset
dataset = pd.read_excel("superstore_data.xls")
print(dataset.head())
print(dataset.info())
print(dataset.describe())
print(dataset.isnull().sum())

#starting from time series analysis and forecasting for furniture sales.
furniture = dataset.loc[dataset['Category'] == 'Furniture']
furniture['Order Date'].min(), furniture['Order Date'].max()

#Data Preprocessing: removing cols we dont need and many more
cols = ['Row ID', 'Order ID', 'Ship Date', 'Ship Mode', 'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State', 'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category', 'Product Name', 'Quantity', 'Discount', 'Profit']
furniture.drop(cols, axis=1, inplace=True)
furniture = furniture.sort_values('Order Date')
furniture.isnull().sum()
furniture = furniture.groupby('Order Date')['Sales'].sum().reset_index()

#stop date: 24/01/2024
#end date:
